In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating environment at `~/planar_quadruped_landing_real_quadruped/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("real_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

WebIO._IJuliaInit()

In [3]:
h = 1
v_init_z = sqrt(2 * 9.81 * h)

4.4294469180700204

In [4]:
# Dynamics model
model = RealQuadruped()

# Some parameters
dt = 0.02
N = 41
times = range(0, dt * (N - 1), length=N)
n = 37
m = 13

# initial state
xinit = zeros(n)

xinit[3] = 0.5 # zb

xinit[7] = 0.15 # x1
xinit[8] = 0.1  # y1

xinit[10] = -0.15 # x2
xinit[11] = 0.1   # y2

xinit[13] = 0.15 # x3
xinit[14] = -0.1 # y3

xinit[16] = -0.15 # x4
xinit[17] = -0.1  # y4

xinit[21] = -v_init_z # vzb

# Desired final state
xterm = zeros(n)

xterm[3] = 0.3 # zb

xterm[7:17] = xinit[7:17];

In [5]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, xterm, dt);

In [6]:
# Objective
Q_diag = 10.0 * ones(n)
Q_diag[end] = 0.0
Q = Diagonal(Q_diag)

R_diag = 1e-3 * ones(m)
R_diag[end] = 0.0
R = Diagonal(R_diag)

Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end

push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [7]:
# Define the NLP
nlp = HybridNLP(model, obj, N, xinit, xterm);

In [8]:
# initialize guess
Xguess = [zeros(n) for x in Xref]

for k = 1:N
    Xguess[k][1:36] = xterm[1:36]
    Xguess[k][end] = dt * (k - 1)
end

mb, g = model.mb, model.g
Uguess = [zeros(m) for u in Uref]

for k = 1:N-1
    Uguess[k][3] = -mb*g/4
    Uguess[k][6] = -mb*g/4
    Uguess[k][9] = -mb*g/4
    Uguess[k][12] = -mb*g/4
    Uguess[k][end] = dt;
end

Z0 = packZ(nlp, Xguess, Uguess);

In [9]:
Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  3497529
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     2037
                     variables with only lower bounds:      160
                variables with lower and upper bounds:    

([-1.0740771157349048e-12, -2.4242161660364105e-11, 0.49999999999994554, -1.283628336706759e-13, -9.38583001177672e-15, -1.7062967182032288e-15, 0.15000000000024596, 0.1000000000075244, -1.0374746893961827e-16, -0.14999999999975433  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5842117694485136], Ipopt.Optimizer)

In [11]:
Z_sol[1:37] - xinit

37-element Vector{Float64}:
 -1.0740771157349048e-12
 -2.4242161660364105e-11
 -5.445643935786393e-14
 -1.283628336706759e-13
 -9.38583001177672e-15
 -1.7062967182032288e-15
  2.4596991110570343e-13
  7.52439777151892e-12
 -1.0374746893961827e-16
  2.456645997739315e-13
  7.5238287822188e-12
 -8.870835041696512e-17
  2.458588888032409e-13
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -7.724812570049827e-18

In [ ]:
Z_sol[end-14:end] - xterm

In [ ]:
@show F1y_final = Z_sol[797]
@show F2y_final = Z_sol[799]
@show F1y_final + F2y_final

In [ ]:
dt = zeros(N - 1)
for k = 1:N-1
    @show dt[k] = Z_sol[20+20*(k-1)];
end

In [ ]:
F1y = zeros(N-1)
F2y = zeros(N-1)
for k = 1:N-1
    @show F1y[k] = Z_sol[17+20*(k-1)];
    @show F2y[k] = Z_sol[19+20*(k-1)];
end

In [ ]:
ui = nlp.uinds
Z_sol[ui[end]][2] + Z_sol[ui[end]][4] + nlp.model.mb * nlp.model.g

In [ ]:
# display(Z_sol)
writedlm("data_1.csv", Z_sol, ',')